In [1]:
from keras.applications import resnet50
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# importing the weights of pre trained model resnet50 but we do not import the output layer of the model
model=resnet50.ResNet50(weights='imagenet' ,input_shape=(224,224,3) , include_top=False)

C:\Users\Ashutosh\anaconda3\envs\tensorflowlab\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
model.layers

In [4]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [5]:
for layer in model.layers:
    layer.trainable = False

In [6]:
#Top layer of the model
model.layers[-1].output

<tf.Tensor 'activation_49/Relu:0' shape=(None, 7, 7, 2048) dtype=float32>

In [7]:
# Adding layer to the model
top_layer = model.output
top_layer = Flatten()(top_layer)
top_layer = Dense(units=1024, activation='relu')(top_layer)
top_layer = Dense(units=1024, activation='relu')(top_layer)
top_layer = Dense(units=512, activation='relu')(top_layer)
top_layer = Dense(units=3, activation='softmax')(top_layer)

In [8]:
from keras.models import Sequential

In [9]:
final_model = Model(inputs = model.input,  output = top_layer)

C:\Users\Ashutosh\anaconda3\envs\tensorflowlab\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [10]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [11]:
final_model.output

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 3) dtype=float32>

In [12]:
#image augumentation for training of the model
train_datagen  = ImageDataGenerator(
                                   rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range  = 0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
                                                'Dataset/Training/',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical'
                                                 )
testing_set = test_datagen.flow_from_directory(
                                              'Dataset/Testing/',
                                               target_size = (224,224),
                                               batch_size = 32,
                                               class_mode = 'categorical'       
                                               )

Found 1163 images belonging to 3 classes.
Found 1135 images belonging to 3 classes.


In [13]:
final_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [15]:
checkpoint = ModelCheckpoint("face_model1.h5",
                           monitor="val_loss",
                           verbose=1,
                           save_best_only=True,
                           mode = "min")

earlystoppage = EarlyStopping(monitor='val_loss',
                           min_delta=0,
                           patience=3,
                           verbose=1,
                           restore_best_weights=True)
# we put our call backs into a callback list
callbacks = [earlystoppage,  checkpoint]

# Enter the number of training and validation samples here
final_train_samples = 1163
final_validation_samples = 1135

# We only train 3 epochs
epochs = 3
batch =16

history =  final_model.fit_generator(
           training_set,
           epochs= epochs,
           steps_per_epoch= final_train_samples//batch,
           callbacks=callbacks,
           validation_data=testing_set,
           validation_steps=final_validation_samples//batch   )


Epoch 1/3
72/72 [==============================] - 1618s 22s/step - loss: 2.3744 - accuracy: 0.9461 - val_loss: 39.4714 - val_accuracy: 0.2937

Epoch 00001: val_loss improved from inf to 39.47143, saving model to face_model1.h5
Epoch 2/3
72/72 [==============================] - 1563s 22s/step - loss: 0.0192 - accuracy: 0.9991 - val_loss: 42.2941 - val_accuracy: 0.2915

Epoch 00002: val_loss did not improve from 39.47143
Epoch 3/3
72/72 [==============================] - 1669s 23s/step - loss: 0.2153 - accuracy: 0.9965 - val_loss: 13.0704 - val_accuracy: 0.4633

Epoch 00003: val_loss improved from 39.47143 to 13.07044, saving model to face_model1.h5


In [16]:
from keras.models import load_model
model = load_model('face_model1.h5')

In [ ]:
# predicting the images from the test data

import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

pred_dict = {"[0]": "Arti",
             "[1]": "Ashu", 
             "[2]": "Mammi",
            }

pred_dict_n = {"Arti ": "Arti", 
               "Ashu": "Ashu",
               "Mammi": "Mammi",
                       }

def draw_test(name, pred, im):
    face_dict = pred_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face_dict, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + pred_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Dataset/Training/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(model.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()


